In [ ]:
!pip install langgraph

In [ ]:
!pip install langchain_google_genai

In [ ]:
!pip install langchain_community pinecone langchain_pinecone

In [86]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentExecutor
from langgraph.graph import StateGraph, START, END
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import ServerlessSpec
from langchain.tools import Tool
import requests

In [ ]:
GEMENI_API_KEY = ''
pinecone_api_key = ''
weather_API_KEY = ''
ALPHA_VANTAGE_APIKEY = ''

In [13]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GEMENI_API_KEY)

In [15]:
#Load text file
loader = TextLoader("Personal_Details.txt")
documents = loader.load()

In [108]:
# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)
index_name = "chatbot-index-gemini"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        # dimension of the vector embeddings produced by gemini-embedding-001
        dimension=768, #dimension=768-small, 1536-medium, 3072-large
        metric="cosine",
        # parameters for the free tier index
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Initialize index client
index = pc.Index(name=index_name)

# View index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 13}},
 'total_vector_count': 13,
 'vector_type': 'dense'}

In [17]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 50)
all_splits = splitter.split_documents(documents)

In [18]:
print(all_splits[0])

page_content='Ajay Bellamkonda, fondly known as Ajay, is a passionate Data Engineer from Hyderabad, India, originally hailing from Challapalli. Born on May 3, 1992, he is an Indian national and a proud Hindu who' metadata={'source': 'Personal_Details.txt'}


In [22]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMENI_API_KEY)

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

ids = [f"doc_{i}" for i in range(len(all_splits))]

vectorstore.add_documents(documents=all_splits, ids=ids)

['doc_0',
 'doc_1',
 'doc_2',
 'doc_3',
 'doc_4',
 'doc_5',
 'doc_6',
 'doc_7',
 'doc_8',
 'doc_9',
 'doc_10',
 'doc_11',
 'doc_12']

In [23]:
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

In [79]:
def personal_info_tool(query : str) -> str :
  """ Retrieve Ajay's information and generate natural answer"""
  results = retriever.invoke(query)
  #print(query)

  if not results:
    return "I am not able to find personal information about that"

  context = "\n".join([doc.page_content for doc in results])
  prompt = f"You are Ajay's assistant. Based on the following context, answer the question:\n\n{context}\n\nQuestion: {query}"
  #print(prompt)
  response = llm.invoke(prompt)

  return response.content

In [91]:
def get_weather_report(location: str) -> str:
    """Fetch current weather for a given location using OpenWeatherMap API."""
    api_key = weather_API_KEY
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"

    response = requests.get(url)
    if response.status_code != 200:
        return f"Couldn't fetch weather for {location}. Please check the location name."

    data = response.json()
    temp = data["main"]["temp"]
    condition = data["weather"][0]["description"]
    return f"The current temperature in {location} is {temp}°C with {condition}."


In [103]:
def get_stock_price(symbol: str) -> str:
    """Fetch the latest stock price for a given symbol using Alpha Vantage."""
    api_key = ALPHA_VANTAGE_APIKEY
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "GLOBAL_QUOTE",
        "symbol": symbol,
        "apikey": api_key
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        return f"Error fetching data for {symbol}. Status code: {response.status_code}"

    data = response.json()
    try:
        quote = data["Global Quote"]
        price = quote["05. price"]
        change = quote["10. change percent"]
        return f"{symbol} is currently trading at ${price} ({change} change)."
    except KeyError:
        return f"No data found for symbol: {symbol}"

In [104]:
#tools = [Tool(name = "PersonalInfoRetriever", func = personal_info_tool,
#              description= ("Use this tool to answer any question about Ajay Bellamkonda, including his education, work experience, personal background, or achievements."))]
PersonalInfoRetriever_tools = Tool(name = "PersonalInfoRetriever", func = personal_info_tool, description= "Use this tool to answer any question about Ajay Bellamkonda, including his education, work experience, personal background, or achievements.")

weather_tool = Tool(name="WeatherFetcher", func=get_weather_report, description="Use this tool to get the current weather for any city.")

stock_tool = Tool(name="StockPriceFetcher", func=get_stock_price, description="Use this tool to get the latest stock price for a company. Input should be a stock symbol like AAPL or TSLA.")

tools = [PersonalInfoRetriever_tools,weather_tool,stock_tool]

In [105]:
agent_executor = initialize_agent(tools=tools, llm=llm)

In [106]:
from typing import TypedDict

class AgentState(TypedDict):
  input : str
  output : str

def call_agent(state:AgentState):
  result = agent_executor.invoke({"input" : state["input"]})
  return {"output" : result["output"]}

workflow = StateGraph(AgentState)
workflow.add_node("agent", call_agent)
workflow.set_entry_point("agent")
workflow.add_edge("agent",END)


app = workflow.compile()

In [107]:
while True:
  query = input("you: ")
  if query.lower() in ["exit","quit"]:
    print("Bot: Good Bye")
    break
  result = app.invoke(({"input": query}))
  print("Bot: ", result["output"])

you: what is the educational qualification of Ajay?
Bot:  Ajay Bellamkonda completed his B.Tech from KKR & KSR Institute of Engineering and Technology, Ganguru, in 2014 with an aggregate of 81%.
you: what is the stock price of MSFT today ?
Bot:  The stock price of MSFT is currently trading at $505.7200 (-0.7945% change).
you: How is the weather in hyderabad today ?
Bot:  The current temperature in Hyderabad is 29.23°C with haze.
you: What is the capital of India ?
Bot:  New Delhi
you: what is Ajay's home town ?
Bot:  Ajay Bellamkonda's hometown is Challapalli.
you: What are his educational qualifications ?
Bot:  Ajay Bellamkonda has a Master’s degree (M.Tech) in Electronics and Communication from IIIT Nuzvidu and a Bachelor's Degree from Engineering and Technology, Ganguru, completed in 2014 with an aggregate of 81%.
you: quit
Bot: Good Bye
